In [ ]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
 
# Download necessary resources
nltk.download('punkt')
nltk.download('stopwords')
 
# Sample text
text = "If only people would just take a step back and not make this case about them, because it wasn't about anyone except the two people in that situation.  To lump yourself into this mess and take matters into your own hands makes these kinds of protests selfish and without rational thought and investigation.  The guy in this video is heavily emotional and hyped up and wants to be heard, and when he gets heard he just presses more and more.  He was never out to have a reasonable discussion.  Kudos to the Smerconish for keeping level the whole time and letting Masri make himself out to be a fool.  How dare he and those that tore that city down in protest make this about themselves and to dishonor the entire incident with their own hate.  By the way, since when did police brutality become an epidemic?  I wish everyone would just stop pretending like they were there and they knew EXACTLY what was going on, because there's no measurable amount of people that honestly witnessed this incident, so none of us have a clue on which way this whole issue should have swung.  The grand jury were the most informed, we have to trust the majority rule was the right course of action and let it be.  Also, thank you to the 99.999% of police officers in America that actually serve & protect, even if you're a bit of a jerk when you pull me over, I respect your job and know that someone has to do it and that many people are going to pout about being held accountable to their actions.  People hate police until they need an officer or two around in an emergency."
 
# Step 1: Sentence tokenization with PUNKT
sentences = sent_tokenize(text)
 
# Step 2: Process each sentence
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))
 
processed_sentences = []
for sentence in sentences:
    # Word tokenization
    words = word_tokenize(sentence)
     
    # Remove stopwords and stem
    filtered_words = [stemmer.stem(word.lower()) for word in words if word.lower() not in stop_words]
     
    processed_sentences.append(filtered_words)
 
print(processed_sentences)

[nltk_data] Downloading package punkt to /Users/trabajo/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True